In [ ]:
# 基本面因子
# 全部资产现金回收率 = 经营性现金流/总资产
cfoa_mrq = Factor("cash_flow_from_operating_activities_mrq_0") / Factor(
    "total_assets_mrq_0"
)
# 资产周转率变动 = 当期营业收入/当期总资产 - 去年同期营业收入/去年同期总资产
atdy_mrq = Factor("operating_revenue_mrq_0") / Factor("total_assets_mrq_0") - Factor(
    "operating_revenue_mrq_4"
) / Factor("total_assets_mrq_4")
## 现金流动负债率比 = 经营性现金流/流动负债
ccr_mrq = Factor("cash_flow_from_operating_activities_mrq_0") / Factor(
    "current_liabilities_mrq_0"
)
## 营业利润同比增长率 = （当期营业利润 - 去年同期营业利润）/绝对值（去年同期营业利润）
op_q_yoy_mrq = (
    Factor("profit_from_operation_mrq_0") - Factor("profit_from_operation_mrq_4")
) / ABS(Factor("profit_from_operation_mrq_4"))
## 股息率
dp_ttm = Factor("dividend_yield_ttm")
## 市盈率
pe_ttm = Factor("market_cap_3") / Factor("net_profit_parent_company_lyr_0")
## 盈市率 = 1/市盈率
ep_ttm = Factor("net_profit_parent_company_lyr_0") / Factor("market_cap_3")
## 营业利润率 = 营业利润/营业收入
opr_mrq = Factor("profit_from_operation_mrq_0") / Factor("operating_revenue_mrq_0")

# 量价因子
HIGH = Factor("high")
LOW = Factor("low")
OPEN = Factor("open")
CLOSE = Factor("close")
VOLUME = Factor("volume")
TURNOVER = Factor("total_turnover")


def turnover_rate(order_book_ids, start_date, end_date):
    return (
        get_turnover_rate(order_book_ids, start_date, end_date, fields="today")
        .today.unstack("order_book_id")
        .reindex(
            columns=order_book_ids,
            index=pd.to_datetime(get_trading_dates(start_date, end_date)),
        )
    )


DAILY_TURNOVER_RATE = UserDefinedLeafFactor("DAILY_TURNOVER_RATE", turnover_rate)

# 流动性因子
## 过去20日换手率的均值
liq_turn_avg = MA(DAILY_TURNOVER_RATE, 20)
## 过去20日换手率的标准差
liq_turn_std = STD(DAILY_TURNOVER_RATE, 20)
## 过去20日成交额的均值/成交额的标准差
liq_zamount = MA(TURNOVER, 20) / STD(TURNOVER, 20)

# 量价相关性
## 过去20天收盘价和日换手率之间的相关性
corr_price_turn = CORR(CLOSE, DAILY_TURNOVER_RATE, 20)

# 波动率
## 过去20天的高低价差比之间的波动率
vol_highlow_std = STD(HIGH / LOW, 20)
## 过去20天上影线的波动率
vol_up_shadow_std = STD((HIGH - MAX(OPEN, CLOSE)) / HIGH, 20)

# 动量
month_day, year_day = 20, 252
# 过去1个月动量
mmt_normal_M = CLOSE / REF(CLOSE, month_day) - 1
# 过去1年动量 （剔除过去1个月动量）
mmt_normal_A = REF(CLOSE, month_day) / REF(CLOSE, year_day) - 1


factor_dict = {
    "cfoa_mrq": cfoa_mrq,
    "atdy_mrq": atdy_mrq,
    "ccr_mrq": ccr_mrq,
    "op_q_yoy_mrq": op_q_yoy_mrq,
    "dp_ttm": dp_ttm,
    "pe_ttm": pe_ttm * -1,
    "ep_ttm": ep_ttm,
    "opr_mrq": opr_mrq,
    "liq_turn_avg": liq_turn_avg * -1,
    "liq_turn_std": liq_turn_std * -1,
    "liq_zamount": liq_zamount,
    "corr_price_turn": corr_price_turn * -1,
    "vol_highlow_std": vol_highlow_std * -1,
    "vol_up_shadow_std": vol_up_shadow_std * -1,
    "mmt_normal_M": mmt_normal_M * -1,
    "mmt_normal_A": mmt_normal_A,
}